In [2]:
# Imports
import sqlite3
import pandas as pd


In [3]:
# Connect to the SQLite database
db_path = r'C:\Users\shaya\OneDrive\Desktop\database\sql-murder-mystery.db'
conn = sqlite3.connect(db_path)

In [4]:
# Define functions for frequently used queries
def get_all_table_names(conn):
    """Retrieve the names of all tables in the database."""
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

def get_foreign_keys_for_table(conn, table_name):
    """Retrieve foreign keys for a specific table."""
    query = f"PRAGMA foreign_key_list({table_name});"
    return pd.read_sql(query, conn)

def get_all_foreign_keys(conn):
    """Retrieve all foreign key relationships in the database."""
    all_foreign_keys = []
    table_names = get_all_table_names(conn)
    
    for table in table_names:
        fk_df = get_foreign_keys_for_table(conn, table)
        if not fk_df.empty:
            fk_df['table'] = table
            all_foreign_keys.append(fk_df)
    
    if all_foreign_keys:
        return pd.concat(all_foreign_keys, ignore_index=True)
    else:
        return pd.DataFrame(columns=['id', 'seq', 'table', 'from', 'to', 'on_update', 'on_delete', 'match'])

def get_all_in_table(conn, table):
    """See everything inside a specific table."""
    query = f"SELECT * from {table}"
    return pd.read_sql(query, conn)


In [5]:
table_names = get_all_table_names(conn)
table_names

['crime_scene_report',
 'drivers_license',
 'person',
 'facebook_event_checkin',
 'interview',
 'get_fit_now_member',
 'get_fit_now_check_in',
 'income',
 'solution']

In [6]:
foreign_keys = get_all_foreign_keys(conn)
foreign_keys

,id,seq,table,from,to,on_update,on_delete,match
0,0,0,person,license_id,id,NO ACTION,NO ACTION,NONE
1,0,0,facebook_event_checkin,person_id,id,NO ACTION,NO ACTION,NONE
2,0,0,interview,person_id,id,NO ACTION,NO ACTION,NONE
3,0,0,get_fit_now_member,person_id,id,NO ACTION,NO ACTION,NONE
4,0,0,get_fit_now_check_in,membership_id,id,NO ACTION,NO ACTION,NONE



""" All we know is murder on Jan.15, 2018 in SQL City """

In [7]:
crime_scene_report = get_all_in_table(conn, 'crime_scene_report')
crime_scene_report.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


## Step 1

In [8]:
# Get all reports for Jan 15 2018 in SQL City
df_crime_report_Jan152018_SQlCity = pd.read_sql("SELECT * FROM crime_scene_report WHERE city = 'SQL City' AND type = 'murder' AND date = '20180115'", conn)
print(df_crime_report_Jan152018_SQlCity.to_string())

       date    type                                                                                                                                                                                description      city
0  20180115  murder  Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".  SQL City


Crime Description: Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

## Step 2

In [9]:
# Check interviews of the witnesses
interviews = get_all_in_table(conn, 'interview')
interviews.head()

,person_id,transcript
0,28508,‘I deny it!’ said the March Hare.\n
1,63713,\n
2,86208,"way, and the whole party swam to the shore.\n"
3,35267,"lessons in here? Why, there’s hardly room for ..."
4,33856,\n


In [10]:
# So we need the person_id of witnesses
# See person table
person = get_all_in_table(conn, 'person')
person.head()

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


Info of 2nd witness

In [11]:
# Check for Annabel living on Franklin Ave
Anabel_FranklinAve = pd.read_sql("SELECT * FROM person WHERE address_street_name = 'Franklin Ave' AND name LIKE 'Annabel%' ", conn)
print(Anabel_FranklinAve.to_string())

      id            name  license_id  address_number address_street_name        ssn
0  16371  Annabel Miller      490173             103        Franklin Ave  318771143


In [12]:
# Check for all houses on Northwestern Dr
all_NorthwesternDr = pd.read_sql("SELECT * FROM person WHERE address_street_name = 'Northwestern Dr' ", conn)
# Make a list of all house numbers 
all_houses_NorthwesternDr = all_NorthwesternDr['address_number'].to_list()
#Find the max (since the witness is reported to live in the last house)
print(max(all_houses_NorthwesternDr))

4919


Info of 1st witness

In [13]:
house4919_NorthwesternDr = pd.read_sql("SELECT * FROM person WHERE address_street_name = 'Northwestern Dr' AND address_number = '4919' ", conn)
house4919_NorthwesternDr

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


## Step3

Now we have the person_id of witnesses so we can check their interviews

In [14]:
witnesses_interview = pd.read_sql("SELECT * FROM interview WHERE person_id = '14887' OR person_id = '16371' ", conn)
print(witnesses_interview.to_string())


   person_id                                                                                                                                                                                                                       transcript
0      14887  I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
1      16371                                                                                                            I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


### Info based on interviews:
Suspect is a male

"Get Fit Now Gym" bag

membership number on the bag started with "48Z"

gold member

plate that included "H42W"

last week on January the 9th



## Step 4
Look for more info in gym memberships 

In [15]:
# See get_fit_now_check_in table
get_fit_now_check_in = get_all_in_table(conn, 'get_fit_now_check_in')
get_fit_now_check_in.head()

,membership_id,check_in_date,check_in_time,check_out_time
0,NL318,20180212,329,365
1,NL318,20170811,469,920
2,NL318,20180429,506,554
3,NL318,20180128,124,759
4,NL318,20171027,418,1019


In [16]:
# Look for membership ID startig with 48Z and check in date Jan 9 2018
membership_id_48Z_20180109 = pd.read_sql("SELECT * FROM get_fit_now_check_in WHERE membership_id LIKE '48Z%' AND check_in_date = '20180109' ", conn)
membership_id_48Z_20180109

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [17]:
# We also know that the suspect is a gold member, we can check membership status from get_fit_now_member
# See get_fit_now_member table
get_fit_now_member = get_all_in_table(conn, 'get_fit_now_member')
get_fit_now_member.head()

,id,person_id,name,membership_start_date,membership_status
0,NL318,65076,Everette Koepke,20170926,gold
1,AOE21,39426,Noe Locascio,20171005,regular
2,2PN28,63823,Jeromy Heitschmidt,20180215,silver
3,0YJ24,80651,Waneta Wellard,20171206,gold
4,3A08L,32858,Mei Bianchin,20170401,silver


In [18]:
membership_status = pd.read_sql("SELECT * FROM get_fit_now_member WHERE id = '48Z7A' OR id = '48Z55' ", conn)
membership_status

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


## Step 5
So far we found 2 potential suspects both having gold membership. One other info we have is that the murderer's plate number includes "H42W" which can be found in drivers_license table.

In [19]:
# See drivers_license table
drivers_license = get_all_in_table(conn, 'drivers_license')
drivers_license.head()

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS


In [20]:
# Look for plate number including "H42W" in males
plate_number_H42W_male = pd.read_sql("SELECT * FROM drivers_license WHERE plate_number LIKE '%H42W%' AND gender = 'male' ", conn)
plate_number_H42W_male

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [21]:
# Compare with person table
person_license_id_423327_664760 = pd.read_sql("SELECT * FROM person WHERE license_id = '423327' OR license_id = '664760' ", conn)
person_license_id_423327_664760

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [24]:
# Compare with get fit now member
membership_status

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold
1,48Z7A,28819,Joe Germuska,20160305,gold


# Conclusion
From the last two tables we can conclude that the murderer is "Jeremy Bowers"!

## Bonus 
We can take this even further!

In [27]:
# look for the interview of the murderer
murderer_interview = pd.read_sql("SELECT * FROM interview WHERE person_id = '67318' ", conn)
print(murderer_interview.to_string())

   person_id                                                                                                                                                                                                                                          transcript
0      67318  I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n


So it looks like there is someone else behind this
Info we get based on the interview

Rich female

5.5 - 5.7 tall

Red hair

Tesla model S car

Attended SQL Symphony Concert 3 times in December 2017

In [46]:
# Check facebook_event_checkin for Dec 2017 and SQL Symphony Concert
facebook_event_checkin_Dec2017_SQLSymphonyConcert = pd.read_sql("SELECT * FROM facebook_event_checkin WHERE date LIKE '201712%' AND event_name = 'SQL Symphony Concert'  ", conn)
facebook_event_checkin_Dec2017_SQLSymphonyConcert

,person_id,event_id,event_name,date
0,62596,1143,SQL Symphony Concert,20171225
1,19260,1143,SQL Symphony Concert,20171214
2,58898,1143,SQL Symphony Concert,20171220
3,69699,1143,SQL Symphony Concert,20171214
4,19292,1143,SQL Symphony Concert,20171213
5,43366,1143,SQL Symphony Concert,20171207
6,92343,1143,SQL Symphony Concert,20171212
7,28582,1143,SQL Symphony Concert,20171220
8,28582,1143,SQL Symphony Concert,20171215
9,81526,1143,SQL Symphony Concert,20171202


In [51]:
# Check drivers_license table with the given information
accomplice_drivers_license = pd.read_sql("SELECT * FROM drivers_license WHERE gender = 'female' AND hair_color = 'red' AND height BETWEEN 65 AND 67 AND car_model = 'Model S' ", conn)
accomplice_drivers_license

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [52]:
# Check for this license ids in person table
person_license_ids = pd.read_sql("SELECT * FROM person WHERE license_id IN ('918773', '291182', '202298')  ", conn)
person_license_ids

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [55]:
# Now we need to look for common person ids in facebook event table we just found above

common_person_ids = []
for id in facebook_event_checkin_Dec2017_SQLSymphonyConcert['person_id']:
    if id in [78881, 90700, 99716]:
        common_person_ids.append(id)

common_person_ids

[99716, 99716, 99716]

## Conclusion 
So 99716 is the person id we are looking for and it has indeed repeated 3 times which shows the suspect attended the event 3 times.

The accomplice or the person behind this murder is Miranda Priestly!

In [56]:
# As the last step let's check her income just for fun since we know she is rich lol!
person_license_ids = pd.read_sql("SELECT * FROM income WHERE ssn = '987756388'  ", conn)
person_license_ids

,ssn,annual_income
0,987756388,310000
